<a href="https://colab.research.google.com/github/Joel-Mk/Finance-Market-Prediction/blob/main/Finance_Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt





In [ ]:
def load_data(ticker="AAPL", period="5y"):
    """Download historical stock data from Yahoo Finance"""
    df = yf.download(ticker, period=period)
    df = df[["Close"]]
    df.dropna(inplace=True)
    return df

In [ ]:
def add_features(df):
    """Add lag features and technical indicators"""
    df["Return"] = df["Close"].pct_change()

    # Lagged Returns (past 1 and 2 days)
    df["Lag1"] = df["Return"].shift(1)
    df["Lag2"] = df["Return"].shift(2)

    # Moving Average (MA5, MA10)
    df["MA5"] = df["Close"].rolling(5).mean()
    df["MA10"] = df["Close"].rolling(10).mean()

    # Volatility (Std dev of returns)
    df["Volatility"] = df["Return"].rolling(10).std()

    df.dropna(inplace=True)
    return df


In [ ]:
def prepare_data(df):
    """Prepare feature matrix X and target y"""
    X = df[["Lag1", "Lag2", "MA5", "MA10", "Volatility"]]
    y = df["Return"]
    return train_test_split(X, y, shuffle=False, test_size=0.2)


In [ ]:
def train_model(X_train, y_train, alpha=1.0):
    """Train Ridge Regression model"""
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    return model


In [ ]:
def evaluate_model(model, X_test, y_test):
    """Evaluate model performance"""
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f" Model Evaluation:\nMSE: {mse:.6f}, R2: {r2:.4f}")
    return y_pred

In [ ]:
def plot_results(y_test, y_pred):
    """Plot actual vs predicted returns"""
    plt.figure(figsize=(12,6))
    plt.plot(y_test.values, label="Actual Returns")
    plt.plot(y_pred, label="Predicted Returns")
    plt.legend()
    plt.title("Financial Market Prediction (Dummy Example)")
    plt.xlabel("Time")
    plt.ylabel("Returns")
    plt.show()

In [ ]:
def run_pipeline(ticker="AAPL", period="5y", alpha=1.0):
    df = load_data(ticker, period)
    df = add_features(df)
    X_train, X_test, y_train, y_test = prepare_data(df)
    model = train_model(X_train, y_train, alpha)
    y_pred = evaluate_model(model, X_test, y_test)
    plot_results(y_test, y_pred)

In [ ]:
if __name__ == "__main__":
    run_pipeline("AAPL", "5y", alpha=0.5)